In [40]:
import os
from dotenv import load_dotenv, find_dotenv
import requests
import pandas as pd
import numpy as np
load_dotenv(find_dotenv())

True

In [9]:
API_KEY = os.environ.get('OPX_KEY')

In [11]:
params = {
    'app_id': API_KEY,
    'symbols': 'CAD,USD',
    'show_alternative': 'true'
}
r = requests.get("https://openexchangerates.org/api/latest.json", params=params)

In [14]:
rates_ = r.json()['rates']
rates_

{'CAD': 1.31015, 'USD': 1}

In [16]:
symbol_from = 'CAD'
symbol_to = 'USD'
value = 3000
value * 1/rates_.get(symbol_from) * rates_.get(symbol_to)

2289.8141434186928

In [26]:
class CurrencyConverter:
    def __init__(self, symbols, API_KEY):
        self.API_KEY = API_KEY
        self.symbols = symbols
        self.symbols_ = ",".join([str(s) for s in symbols])
        params = {
            'app_id': API_KEY,
            'symbols': self.symbols_,
            'show_alternative': 'true'
        }
        r = requests.get("https://openexchangerates.org/api/latest.json", params=params)
        self.rates_ = r.json()['rates']
        self.rates_['USD'] = 1
        
    def convert(self, value, symbol_from, symbol_to, round_output=True):
        try:
            x = (value
                * 1/self.rates_.get(symbol_from)
                * self.rates_.get(symbol_to))
            if round_output:
                return round(x, 2)
            else:
                return x
        except TypeError as e:
            print(e)
            print('Unavailable or invalid symbol')
            return None

In [29]:
API_KEY = os.environ.get("OPX_KEY")
c = CurrencyConverter(['CAD', 'USD'], API_KEY)
print(c.convert(3000, 'CAD', 'USD'))
print(c.convert(5000, 'USD', 'CAD'))

2289.82
6550.72


In [30]:
c = CurrencyConverter(['CAD', 'USD', 'DOGE', 'ETH', 'BTC'], API_KEY)
c.rates_

{'BTC': 0.000128346064,
 'CAD': 1.310144,
 'DOGE': 320.7432225,
 'ETH': 0.0019250019,
 'USD': 1}

In [34]:
c.convert(3000, 'CAD', 'ETH')

4.41

In [38]:
df = pd.read_excel('./data/xirr.xlsx', sheet_name='irregular')
df['total'] = df.income + df.expenses
df.head()

,date,income,expenses,total
0,2017-01-01,40,-3000,-2960
1,2017-01-25,40,-50,-10
2,2017-02-12,80,-50,30
3,2017-02-14,100,-30,70
4,2017-03-04,100,-20,80


In [39]:
df['total'].apply(lambda x: c.convert(x, 'CAD', 'USD'))

0    -2259.29
1       -7.63
2       22.90
3       53.43
4       61.06
5       99.23
6       91.59
7       76.33
8       30.53
9      114.49
10     244.25
11      99.23
12     167.92
13     282.41
14     290.04
15     228.98
16     244.25
17     381.64
Name: total, dtype: float64

In [43]:
rate = 0.0575
term = 14
loan= 3000.0
payment = np.round(-np.pmt(rate/12, term, loan), 2)
payment

222.07

In [45]:
balance = loan
df = pd.DataFrame({
    'month': [0],
    'payment': np.NaN,
    'principal': np.NaN,
    'balance': [balance]
})
print(df)

   balance  month  payment  principal
0   3000.0      0      NaN        NaN


In [46]:
interest = round(rate/12 * balance, 2)
principal = payment - interest
balance = balance - principal

In [48]:
print(interest)
print(principal)
print(balance)

14.38
207.69
2792.31


In [51]:
balance = loan
for i in range(1, term + 1):
    interest = round(rate/12 * balance, 2)
    principal = payment - interest
    balance = balance - principal
    df = df.append(
        pd.DataFrame({
            'month': [i],
            'payment': [payment],
            'interest': [interest],
            'principal': [principal],
            'balance': [balance]
    }) )

In [52]:
df = df.reset_index(drop=True)
df[['month', 'payment', 'interest', 'principal', 'balance']]

,month,payment,interest,principal,balance
0,0,NaN,NaN,NaN,3000.00
1,1,222.07,14.38,207.69,2792.31
2,2,222.07,13.38,208.69,2583.62
3,3,222.07,12.38,209.69,2373.93
4,4,222.07,11.38,210.69,2163.24
5,5,222.07,10.37,211.70,1951.54
6,6,222.07,9.35,212.72,1738.82
7,7,222.07,8.33,213.74,1525.08
8,8,222.07,7.31,214.76,1310.32
9,9,222.07,6.28,215.79,1094.53


In [58]:
balance = loan
index = range(0, term + 1)
columns = ['month', 'payment', 'interest', 'principal', 'balance']
df = pd.DataFrame(index=index, columns=columns)

In [59]:
df.iloc[0]['month'] = 0
df.iloc[0]['balance'] = balance

In [60]:
for i in range(1, 11):
    interest = round(rate/12 * balance, 2)
    principal = payment - interest
    balance = balance - principal
    df.iloc[i]['month'] = i
    df.iloc[i]['payment'] = payment
    df.iloc[i]['interest'] = interest
    df.iloc[i]['principal'] = principal
    df.iloc[i]['balance'] = balance
df

,month,payment,interest,principal,balance
0,0,NaN,NaN,NaN,3000
1,1,222.07,14.38,207.69,2792.31
2,2,222.07,13.38,208.69,2583.62
3,3,222.07,12.38,209.69,2373.93
4,4,222.07,11.38,210.69,2163.24
5,5,222.07,10.37,211.7,1951.54
6,6,222.07,9.35,212.72,1738.82
7,7,222.07,8.33,213.74,1525.08
8,8,222.07,7.31,214.76,1310.32
9,9,222.07,6.28,215.79,1094.53


In [63]:
def am(loan, rate, term):
    payment = np.round(-np.pmt(rate/12, term, loan), 2)
    balance = loan
    index = range(0, term + 1)
    columns = ['month', 'payment', 'interest', 'principal', 'balance']
    df = pd.DataFrame(index=index, columns=columns)
    df.iloc[0]['month'] = 0
    df.iloc[0]['balance'] = balance
    for i in range(1, term + 1):
        interest = round(rate/12 * balance, 2)
        principal = payment - interest
        balance = balance - principal
        df.iloc[i]['month'] = i
        df.iloc[i]['payment'] = payment
        df.iloc[i]['interest'] = interest
        df.iloc[i]['principal'] = principal
        df.iloc[i]['balance'] = balance
    return df

In [64]:
loan = 3000
gringotts = am(loan, 0.0575, 14)
goliath = am(loan, 0.0399, 20)
gotham = am(loan, 0.0889,  8)

In [65]:
print(gotham['interest'].sum())
print(goliath['interest'].sum())
print(gringotts['interest'].sum())

100.88
105.82999999999997
108.93999999999998
